In [1]:
# -*- coding: utf-8 -*-
#  Stage 1: Data Ingestion and Processing

import os
import pandas as pd
import shutil
from datetime import datetime  # استيراد مكتبة التاريخ

print("--- Stage 1: Data Ingestion and Processing ---")

# تحديد التاريخ الحالي لاستخدامه في مسار Raw Zone
today_date = datetime.now().strftime('%Y%m%d')

# تحديد المسارات الخاصة بمناطق البيانات
raw_zone_base = 'data_lake/raw_zone/student_performance'
raw_zone_path = os.path.join(raw_zone_base, today_date)

processed_zone_path = 'data_lake/processed_zone'
curated_zone_path = 'data_lake/curated_zone'

# إنشاء المجلدات (إذا لم تكن موجودة)
os.makedirs(raw_zone_path, exist_ok=True)
os.makedirs(processed_zone_path, exist_ok=True)
os.makedirs(curated_zone_path, exist_ok=True)

print(" Directories created successfully:")
print(f"  - Raw Zone: {raw_zone_path}")
print(f"  - Processed Zone: {processed_zone_path}")
print(f"  - Curated Zone: {curated_zone_path}")


--- Stage 1: Data Ingestion and Processing ---
 Directories created successfully:
  - Raw Zone: data_lake/raw_zone/student_performance/20251024
  - Processed Zone: data_lake/processed_zone
  - Curated Zone: data_lake/curated_zone


In [2]:
from google.colab import files

uploaded = files.upload()

original_file_name = 'student_performance_dashboard.csv'


Saving student_performance_dashboard.csv to student_performance_dashboard.csv


In [3]:
print("[1/4] Ingesting raw data into the Raw Zone...")

# التحقق من وجود الملف ثم نسخه إلى Raw Zone
if os.path.exists(original_file_name):
    shutil.copy(original_file_name, os.path.join(raw_zone_path, original_file_name))
    print(f" Successfully copied '{original_file_name}' to Raw Zone in the dated path: {raw_zone_path}")
else:
    print(f" ERROR: File '{original_file_name}' not found. Please upload it first.")
    raise FileNotFoundError(f"File '{original_file_name}' not found.")


[1/4] Ingesting raw data into the Raw Zone...
 Successfully copied 'student_performance_dashboard.csv' to Raw Zone in the dated path: data_lake/raw_zone/student_performance/20251024


In [4]:
print("[2/4] Reading from Raw Zone and initial processing...")

# تحديد المسار الكامل للملف داخل Raw Zone
csv_path = os.path.join(raw_zone_path, original_file_name)

# قراءة البيانات
df = pd.read_csv(csv_path)

# تحويل عمود تاريخ الامتحان إلى نوع datetime
df['final_exam_date'] = pd.to_datetime(df['final_exam_date'])

# ملء القيم المفقودة في الأعمدة الرقمية بالـ median
numeric_cols = df.select_dtypes(include=['number']).columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

print(f" Data loaded. Columns: {df.columns.tolist()}")
print(f"Rows: {len(df)}")


[2/4] Reading from Raw Zone and initial processing...
 Data loaded. Columns: ['student_id', 'name', 'age', 'gender', 'grade_level', 'subject_name', 'teacher_name', 'difficulty_level', 'hours_per_week', 'description', 'month', 'score', 'sleep_hours', 'study_hours', 'attendance_rate', 'internet_access', 'free_time_activity', 'parent_education', 'homework_completion_rate', 'extracurricular_activities', 'family_size', 'previous_gpa', 'school_transport', 'health_condition', 'tutoring', 'student_status', 'admission_year', 'city', 'country', 'scholarship_status', 'exam_attempts', 'final_exam_date', 'teacher_experience_years', 'school_branch', 'school_type', 'course_fee', 'payment_status', 'parent_income', 'device_used', 'feedback_rating', 'efficiency', 'sleep_study_ratio', 'attendance_category', 'performance_level', 'exam_month_num', 'exam_final_month', 'days_between_exam_and_month']
Rows: 1000


In [5]:
print("\n[3/4] Applying Data Quality Checks (DQC)...")

initial_rows = len(df) # عدد الصفوف الكلي قبل أي DQC

# 1. التحقق من التكرارات الحقيقية (نفس الطالب، المادة، الشهر)
unique_columns = ['student_id', 'subject_name', 'month']
initial_rows_before_dedup = len(df)
print(f" INFO: Rows before checking duplicates based on {unique_columns}: {initial_rows_before_dedup}")
df.drop_duplicates(subset=unique_columns, keep='first', inplace=True)
rows_after_dedup = len(df)
removed_duplicates = initial_rows_before_dedup - rows_after_dedup
if removed_duplicates > 0:
    print(f" INFO: Removed {removed_duplicates} fully duplicate rows based on {unique_columns}.")
else:
    print(f" INFO: No fully duplicate rows found based on {unique_columns}.")

# 2. التحقق من أن درجات الطلاب بين 0 و 100
df = df[(df['score'] >= 0) & (df['score'] <= 100)]
rows_after_score_check = len(df)
removed_score_issues = rows_after_dedup - rows_after_score_check
if removed_score_issues > 0:
    print(f" WARNING: Removed {removed_score_issues} rows due to score out of range [0-100].")
else:
    print(f" INFO: Score range check passed. All scores are within [0-100].")


# حساب إجمالي الصفوف المحذوفة في كل خطوات DQC
total_removed_rows = initial_rows - len(df)

if total_removed_rows > 0:
    print(f" DQC Summary: Total removed rows = {total_removed_rows}.")
else:
    print(" DQC Passed: All quality checks met without removing rows.")

print(f"Columns after DQC: {df.columns.tolist()}")
print(f"Rows after Processing and DQC: {len(df)}") # عدد الصفوف النهائي بعد كل DQC


[3/4] Applying Data Quality Checks (DQC)...
 INFO: Rows before checking duplicates based on ['student_id', 'subject_name', 'month']: 1000
 INFO: No fully duplicate rows found based on ['student_id', 'subject_name', 'month'].
 INFO: Score range check passed. All scores are within [0-100].
 DQC Passed: All quality checks met without removing rows.
Columns after DQC: ['student_id', 'name', 'age', 'gender', 'grade_level', 'subject_name', 'teacher_name', 'difficulty_level', 'hours_per_week', 'description', 'month', 'score', 'sleep_hours', 'study_hours', 'attendance_rate', 'internet_access', 'free_time_activity', 'parent_education', 'homework_completion_rate', 'extracurricular_activities', 'family_size', 'previous_gpa', 'school_transport', 'health_condition', 'tutoring', 'student_status', 'admission_year', 'city', 'country', 'scholarship_status', 'exam_attempts', 'final_exam_date', 'teacher_experience_years', 'school_branch', 'school_type', 'course_fee', 'payment_status', 'parent_income', '

In [6]:
print("\n[4/4] Saving cleaned data to Processed Zone as Parquet file...")

# تحديد مسار الملف الناتج
processed_file_path = os.path.join(processed_zone_path, 'students_processed.parquet')

# حفظ البيانات في ملف parquet
df.to_parquet(processed_file_path, index=False, engine='pyarrow')

print(f" Stage 1 Complete! Cleaned data has been saved to: {processed_file_path}")
print(" يمكنك الآن العثور على الملف في data_lake/processed_zone على اليسار.")



[4/4] Saving cleaned data to Processed Zone as Parquet file...
 Stage 1 Complete! Cleaned data has been saved to: data_lake/processed_zone/students_processed.parquet
 يمكنك الآن العثور على الملف في data_lake/processed_zone على اليسار.


In [8]:
from google.colab import files

# تحميل الملف النهائي
files.download('data_lake/processed_zone/students_processed.parquet')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>